# What to change? (minimum)
*   Paths replacement at 3 palces for FeatureCollection aka fc: `projects/user-project/assets/SLO-part1`\
*   Change file name where saving (Slovenia#No or 33TUL): `with open(f'Slovenia1/%s.txt' % fc.getInfo().get('features')[0].get('properties').get('ID'), 'w') as f:`\
*   Change filtering of granules in s2 imageCollection
*   In Downlaod change filenames according to granule


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import ee
import google
import os

import json
!pip install retry
from retry import retry

In [ ]:
from google.colab import drive, files
drive.mount('/content/gdrive/')

In [ ]:
%cd /content/gdrive/MyDrive/PhD_p2
%pwd

In [ ]:
COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS = None

import ee
import google
import os

if COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS is None:
  print("Authenticating using Notebook auth...")
  if os.path.exists(ee.oauth.get_credentials_path()) is False:
    ee.Authenticate()
  else:
    print('\N{check mark} '
          'Previously created authentication credentials were found.')
  ee.Initialize(project='user-project', opt_url='https://earthengine-highvolume.googleapis.com') # user project name
else:
  print('Authenticating using Colab auth...')
  # Authenticate to populate Application Default Credentials in the Colab VM.
  google.colab.auth.authenticate_user()
  # Create credentials needed for accessing Earth Engine.
  credentials, auth_project_id = google.auth.default()
  # Initialize Earth Engine.
  ee.Initialize(credentials, project=COLAB_AUTH_FLOW_CLOUD_PROJECT_FOR_API_CALLS, opt_url='https://earthengine-highvolume.googleapis.com')
print('\N{check mark} Successfully initialized!')

In [ ]:
fc = ee.FeatureCollection('projects/user-project/assets/S133TXM')

def getRequests():
  """Generates a list of work items to be downloaded.
  Produces 1000 random points in each of the RESOLVE ecoregions in the ROI.
  """
  fc = ee.FeatureCollection('projects/user-project/assets/S133TXM')

  return fc.aggregate_array('system:index').getInfo()

print(getRequests())

In [ ]:
# For Sentinel-2 processing

# // Calcaulate NDVI
def add_ndvi(img):
  ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI').copyProperties(img, ['system:time_start']);
  return(img.addBands(ndvi));

# //Calculate EVI
def add_evi(img):
  evi=img.expression('2.5 * (NIR - RED) / ((NIR + 6*RED - 7.5*BLUE) + 1)',{
  'BLUE':img.select('B2'),
  'RED':img.select('B4'),
  'NIR':img.select('B8')}).rename('EVI');
  return(img.addBands(evi));

# //Calculate EVI2
def add_evi2(img):
  evi2=img.expression('2.5 * (NIR - RED) / (NIR + 2.4*RED + 1)',{
  'BLUE':img.select('B2'),
  'RED':img.select('B4'),
  'NIR':img.select('B8')}).rename('EVI2');
  return(img.addBands(evi2));

# //Calculate NDMI
def add_ndmi(img):
  ndmi=img.expression('(NIRnarrow - SWIR)/(NIRnarrow + SWIR)',{
  'SWIR':img.select('B11'),
  'NIRnarrow':img.select('B8A')}).rename('NDMI');
  return(img.addBands(ndmi));

# //Calculate SAVI
def add_savi(img):
  savi=img.expression('( (NIR - RED) / (NIR + RED + 0.5) ) * (1 + 0.5)',{
  'RED':img.select('B4'),
  'NIR':img.select('B8')}).rename('SAVI');
  return(img.addBands(savi));

# //Calculate PSRI (Plant senescence reflectance index)
def add_psri(img):
  psri=img.expression('(RED - BLUE) / Re2',{
  'BLUE':img.select('B2'),
  'RED':img.select('B4'),
  'Re2':img.select('B6')}).rename('PSRI');
  return(img.addBands(psri));

# //Calculate SeLI
def add_seli(img):
  seli=img.expression('(NIRnarrow - RedEdgeI) / (NIRnarrow + RedEdgeI)',{
  'RedEdgeI':img.select('B5'),
  'NIRnarrow':img.select('B8A')}).rename('SeLI');
  return(img.addBands(seli));

# //Calculate ARVI
def add_arvi(img):
  arvi=img.expression('(NIR - RED - 1 * (RED - BLUE)) / (NIR + RED - 1 * (RED - BLUE))',{
  'BLUE':img.select('B2'),
  'RED':img.select('B4'),
  'NIR':img.select('B8')}).rename('ARVI');
  return(img.addBands(arvi));

# //Calculate ExG
def add_exg(img):
  exg=img.expression('2 * GREEN - RED - BLUE',{
  'BLUE':img.select('B2'),
  'GREEN':img.select('B3'),
  'RED':img.select('B4')}).rename('ExG');
  return(img.addBands(exg));

# //Calculate Chl-red-edge
def add_chlre(img):
  chlre=img.expression('(RedEdgeIII/RedEdgeI) - 1',{
  'RedEdgeI':img.select('B5'),
  'RedEdgeIII':img.select('B7')}).rename('Chl-red-edge');
  return(img.addBands(chlre));

In [ ]:
# For Sentinel-1 processing

# //Calculate Simple Ratio (SR)
def add_sr(img):
  sr=img.expression('VH/VV',{
  'VH':img.select('VH'),
  'VV':img.select('VV')}).rename('SR');
  return(img.addBands(sr));

In [ ]:
# For Sentinel-2 processing, but not used in manuscript. In manuscript we used GEE's Cloud Score

# // Function to mask non-vegetation pixels
def maskNonVegetation(image):
  # // Select the SCL band
  scl = image.select('SCL')
  # // Create a mask for vegetation class (value 4 for vegetation)
  vegetationMask = scl.eq(4)
  # // Update the image mask
  return image.updateMask(vegetationMask)

In [ ]:
# For Sentinel-2 processing in PhD_2.

# Dealing with clouds (Google's product CLOUD SCORE)
def csscore(img):
  csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
  QA_BAND = 'cs_cdf'
  CLEAR_THRESHOLD = 0.65
  score = img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD))
  return(score)

In [ ]:
!git clone https://github.com/adugnag/gee_s1_ard.git  /content/gdrive/MyDrive/PhD_p2/gee_s1_ard/

In [ ]:
%cd /content/gdrive/MyDrive/PhD_p2/gee_s1_ard/python-api

In [ ]:
import wrapper as wp

@retry(tries=10, delay=1, backoff=2)
def getResult(index, fc):
  """Handle the HTTP requests to download an image."""

  fc = ee.FeatureCollection('projects/user-project/assets/S133TXM').filter(ee.Filter.eq('system:index', fc))

  # print(fc.getInfo().get('features')[0].get('properties').get('OBJECTID'))
  print(str(fc.first().getInfo().get('properties').get('ID')))

  # # # Using this only if it breaks or stops workng so that it doesn't go from beginning, but only process the remaining missing.
  # if os.path.isfile(f'/content/gdrive/MyDrive/PhD_p2/S133TXM/%s.txt' % fc.getInfo().get('features')[0].get('properties').get('ID')):
  #   print('Done')
  #   return


  # # # When dealing with Sentinel-1

  #Parameters
  parameter = {  'START_DATE': '2021-04-01',
              'STOP_DATE': '2021-09-30',
              'POLARIZATION': 'VVVH',
              'ORBIT' : 'BOTH',
              'ROI': fc.geometry(),
              'APPLY_BORDER_NOISE_CORRECTION': True,
              'APPLY_SPECKLE_FILTERING': True,
              'SPECKLE_FILTER_FRAMEWORK':'MONO',
              'SPECKLE_FILTER': 'LEE',
              'SPECKLE_FILTER_KERNEL_SIZE': 5,
              'SPECKLE_FILTER_NR_OF_IMAGES':10,
              'APPLY_TERRAIN_FLATTENING': True,
              'DEM': ee.Image('USGS/SRTMGL1_003'),
              'TERRAIN_FLATTENING_MODEL': 'VOLUME',
              'TERRAIN_FLATTENING_ADDITIONAL_LAYOVER_SHADOW_BUFFER':0,
              'FORMAT': 'DB',
              'CLIP_TO_ROI': False,
              'SAVE_ASSET': False,
              'ASSET_ID': "users/amullissa"
              }
  #processed s1 collection
  s1_processed = wp.s1_preproc(parameter).select(['VH','VV']).map(add_sr)

  tabledata = s1_processed.toBands().reduceRegion(reducer=ee.Reducer.median(), geometry=fc.geometry(), scale=10, bestEffort=True, maxPixels=15000000).getInfo()



  # # # When dealing with Sentinel-2 (below)


  # # Dealing with clouds (Google's product CLOUD SCORE)
  # csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
  # QA_BAND = 'cs_cdf'

  # s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-04-01','2021-09-30')\
  # .filterMetadata('MGRS_TILE', 'EQUALS','34TDS')\
  # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',100)).filterBounds(fc.geometry())\
  # .select(['SCL','B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']).map(add_ndvi).map(add_evi).map(add_evi2)\
  # .map(add_ndmi).map(add_savi).map(add_psri).map(add_seli).map(add_arvi).map(add_exg).map(add_chlre).linkCollection(csPlus, [QA_BAND]).map(csscore)\
  # .select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12','NDVI','EVI','EVI2','NDMI','SAVI','PSRI','SeLI','ARVI','ExG','Chl-red-edge']);

  # s2.getInfo()

  # # tabledata = s2.toBands().sample(region=fc.geometry(), scale=10, dropNulls=False).getInfo()
  # tabledata = s2.toBands().reduceRegion(reducer=ee.Reducer.median(), geometry=fc.geometry(), scale=10, bestEffort=True, maxPixels=15000000).getInfo()


  # Saving either for S1 or S2.
  with open(f'/content/gdrive/MyDrive/PhD_p2/S133TXM/%s.txt' % fc.getInfo().get('features')[0].get('properties').get('ID'), 'w') as f:
    f.write(str(tabledata))

In [ ]:
import multiprocessing

items = getRequests()

pool = multiprocessing.Pool(25)
pool.starmap(getResult, enumerate(items))
pool.close()
pool.join()

print('Finsihed')

# Donwload of small .txt file from Google Drive

In [ ]:
!zip -r '/content/S133TXM.zip' '/content/gdrive/My Drive/PhD_p2/S133TXM'

In [ ]:
files.download("/content/S133TXM.zip")

### Checking number of files in a directory (folder) on Google drive and size of the folder.

In [ ]:
import os
# Specify the directory containing the files
directory = '/content/gdrive/My Drive/PhD_p2/S133TXM'

# List all files in the directory
files = os.listdir(directory)

# Count the number of files
number_of_files = len(files)

print(f'Total number of files: {number_of_files}')

In [ ]:
import os

def get_folder_size(directory):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

# Specify the directory containing the files
directory = 'Slovenia'

# Calculate the total size of the folder
folder_size = get_folder_size(directory)

# Convert size to megabytes (optional)
folder_size_mb = folder_size / (1024 * 1024)

print(f'Total size of the folder: {folder_size_mb:.2f} MB')

# Some useful links while producing code

In [ ]:
# https://gis.stackexchange.com/questions/344783/gee-create-imagecollection-of-a-specific-tile
"""
I think the you need to use the MGRS_TILE property to filter your collection metadata on:

var dataset = ee.ImageCollection('COPERNICUS/S2')
                  .filterDate('2019-10-14', '2019-12-08')
                  .filterMetadata('MGRS_TILE', 'EQUALS','48QTD')
Confusing bit is your property is probably 48QTD and not T48QTD?
"""
# https://datascience.stackexchange.com/questions/63855/how-to-load-numerous-files-from-google-drive-into-colab